In [1]:
import re
import os

#%pip install pandas
import pandas as pd

#%pip install --upgrade pip

#%pip install requests
import requests

#%pip install bs4
from bs4 import BeautifulSoup


#this is how I imported the library i've been using to extract keywords
#I have not been able to get it to work yet (see extract_keywords_from_text function)
#if anyone has suggestions or other libraries that might work better, please let me know!

#this was copied from https://stackoverflow.com/questions/38916452/nltk-download-ssl-certificate-verify-failed
#correcting an error that sometimes occurs when downloading nltk functions

#%pip install rake-nltk
from rake_nltk import Rake
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()
nltk.download('punkt_tab')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/echoes/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
def urls_in_file(file_name):
    """ Takes an .eml file name as a string and returns a list of all unique urls linking to news sources """

    #reads the file and removes newlines and equal signs (which are placed at the end of each line)
    with open(file_name, 'r') as file:
        textfile = file.read().replace('\n', '').replace('=', '')
    
    #any strings without white spaces starting with 3Dhttps:// and ending with &amp, non-greedy matching
    valid_urls = re.findall('3D(https://\S*?)?&amp', textfile)
    
    #creates a list of each unique url (removes duplicates)
    unique_urls = list(set(valid_urls))
    
    return unique_urls

#demonstrates the function with the first file in the folder
#urls_in_file('eml_files/1. El Salvador Mineria 1-94 eml/Google Alert - El Salvador minería_1.eml')

In [3]:
def urls_in_folder(folder_name):
    """ Takes a folder of .eml files as a string and returns a list of all unique urls contained in each of the 
    files in the folder 
"""
    
    #creates a list of all file names in folder_name
    filenames = os.listdir(folder_name)
    filenames = [os.path.join(folder_name, file) for file in filenames]
    
    #creates a list to store the unique urls found in each file using urls_in_file function
    all_urls = []
    for file in filenames:
        urls = urls_in_file(file)
        all_urls.extend(urls)
    
    return all_urls

#demonstrates the function with folder '1. El Salvador Mineria 1-94 eml'
#urls_in_folder('eml_files/1. El Salvador Mineria 1-94 eml')[:7]

## Helpful Utility Functions 

**Try to integrate these functions into your code.** 

### Enhanced header date parsing

In [1]:
from email.utils import parsedate_to_datetime
from datetime import timezone 

def extract_date_header(msg):
    # Need to parse the Date header from Google Alert 
    date_header = msg.get("Date")

    if not date_header:
        print("[WARN] date was not extracted.")
        return None 

    try:
        dt = parsedate_to_datetime(date_header) # try to parse the header into a datetime object 
        if dt is None:
            print("[WARN] date header not able to turn into datetime object")
            return None 
        return dt.astimezone(timezone.utc).isoformat()
    except Exception:
        print("Exception occurred!")
        return None 

### Unwrap Google redirect URLs 

Google will give you URLs that look like: 

*www.google.com/url?...&url=www.example.com* 

The unwrap_google_url() function below will extract that "www.example.com" from the whole Google redirect URL. 

In [2]:
from urllib.parse import urlparse, parse_qs, unquote 

def unwrap_google_url(google_redirect):
    if not google_redirect:
        return google_redirect
    
    parsed = urlparse(google_redirect)    # this breaks URL into components 

    if parsed.netloc.endswith("google.com") and parsed.path.strip("/").lower() == "url":
        # if above returns True, then it is google redirect 
        url_query = parse_qs(parsed.query)

        if "url" in url_query and url_query["url"]:
            return unquote(url_query["url"][0])
    return google_redirect   # if it is not a wrapper for another URL, then just return it (because it is not redirect!)

### Get (plan_text, html_text) from Email Message 

In [ ]:
def get_bodies(msg):
    """Return (plain_text, html_text) from an EmailMessage; warn if missing."""
    plain_text, html_text = None, None
    try:
        p = msg.get_body(preferencelist=('plain',))
        if p: plain_text = p.get_content()
    except Exception as e:
        print(f"[WARN] plain body decode failed: {e}")
    try:
        h = msg.get_body(preferencelist=('html',))
        if h: html_text = h.get_content()
    except Exception as e:
        print(f"[WARN] html body decode failed: {e}")
    if plain_text is None: print("[WARN] No text/plain part found.")
    if html_text  is None: print("[WARN] No text/html part found.")
    return plain_text, html_text



### Get inboxmarkup JSON 

JSON = javascript object notation. It is extremely widely used as a file type and data type. It integrates very well with many languages, including Python (and obviously Javascript as well). 

In [ ]:
import json 

def extract_inboxmarkup_json(html):
    """Return dict from <script data-scope='inboxmarkup' type='application/json'>, else None."""
    if not html:
        print("[WARN] No HTML body; cannot extract inboxmarkup JSON.")
        return None
    m = re.search(
        r'<script[^>]*data-scope=["\']inboxmarkup["\'][^>]*type=["\']application/json["\'][^>]*>\s*({.*?})\s*</script>',
        html, flags=re.IGNORECASE | re.DOTALL
    )
    if not m:
        print("[WARN] inboxmarkup JSON not found in HTML.")
        return None
    try:
        return json.loads(m.group(1))
    except json.JSONDecodeError:
        try:
            return json.loads(m.group(1).replace("\\u0026", "&"))
        except Exception as e:
            print(f"[WARN] inboxmarkup JSON parse failed: {e}")
            return None

### EML file parser 

This is a major function that will help a lot! 

In [ ]:
from email import policy
from email.parser import BytesParser
from email.utils import parsedate_to_datetime
from datetime import timezone
import pandas as pd

def parse_eml_file(eml_path):
    """Return {'email_meta': {...}, 'items': [...] } for one .eml file."""
    with open(eml_path, "rb") as f:
        msg = BytesParser(policy=policy.default).parse(f)

    # date header → ISO UTC
    raw_date = msg.get("Date")
    date_iso = None
    if raw_date:
        try:
            date_iso = parsedate_to_datetime(raw_date).astimezone(timezone.utc).isoformat()
        except Exception:
            print(f"[WARN] Could not parse Date header: {raw_date!r}")
    else:
        print("[WARN] Date header missing.")

    # bodies + inboxmarkup
    plain_text, html_text = get_bodies(msg)
    inbox = extract_inboxmarkup_json(html_text or "")

    # items
    items = []
    entity_title, latest_subtitle, latest_iso = None, None, None
    if inbox:
        ent = inbox.get("entity") or {}
        entity_title = ent.get("title")
        latest_subtitle = ent.get("subtitle")
        if latest_subtitle:
            # e.g., "Latest: March 31, 2025"
            raw = latest_subtitle.split(":", 1)[-1].strip()
            ts = pd.to_datetime(raw, errors="coerce")
            if pd.notna(ts):
                latest_iso = (ts.tz_localize("UTC").isoformat()
                              if ts.tzinfo is None else ts.tz_convert("UTC").isoformat())
        for card in inbox.get("cards", []):
            for w in card.get("widgets", []):
                if w.get("type") == "LINK":
                    g = w.get("url")
                    r = unwrap_google_url(g) if g else None
                    items.append({
                        "eml_path": str(eml_path),
                        "message_id": msg.get("Message-ID"),
                        "email_date_utc": date_iso,
                        "json_latest_date_utc": latest_iso,
                        "entity_title": entity_title,
                        "item_title": w.get("title"),
                        "item_description": w.get("description"),
                        "google_url": g,
                        "resolved_url": r
                    })
    print(f"[INFO] Parsed {eml_path}: {len(items)} items.")
    return {
        "email_meta": {
            "eml_path": str(eml_path),
            "message_id": msg.get("Message-ID"),
            "subject": msg.get("Subject"),
            "from": msg.get("From"),
            "to": msg.get("To"),
            "list_id": msg.get("List-Id"),
            "date_email_raw": raw_date,
            "date_email_utc": date_iso,
            "entity_title": entity_title,
            "json_latest_subtitle": latest_subtitle,
            "json_latest_date_utc": latest_iso
        },
        "items": items
    }


## Ideas for language processing 

- simple bilingual lexicons 


In [ ]:
# ... 

In [4]:
def extract_keywords_from_eml(file_name):
    """ 
    Takes an .eml file name as a string and returns a list of the keywords used in the Google Alerts
    that generated the file. 

    Only works with Google Alert .eml files with an ENGLISH subject line, which are formatted as:
    Subject: Google Alert - "keyword1" "keyword2" "keyword3"

    Google Alerts in Spanish appear to be formatted with UTF-8 encoding, which results in subject lines like:
    Subject: =?UTF-8?Q?Google_Alert_=2D_El_Salvador_miner=C3=ADa?=

    NOTE: I chose to use the actual eml file text rather than the name of the file because I didn't want
    the function to be dependent on the file name format, which might vary in the future.
    """   
    
    #reads the file and removes equal signs (which are placed at the end of each line)
    with open(file_name, 'r') as file:
        textfile = file.read().replace('=', '')
    
    #creates a list of keywords found in the textfile
    pattern = 'Subject: Google Alert - (.*)' 
    keyword_block = re.findall(pattern, textfile)
    keywords = keyword_block[0].split('" "')
    keywords = [keywords[i].replace('"', '').replace("'", "") for i in range(len(keywords))]
    
    return keywords

#demonstrates function with Google Alert - Bitcoin_ _El Salvador__1.eml
extract_keywords_from_eml('eml_files/Google Alert - _Bitcoin_ _El Salvador__1.eml')

['Bitcoin', 'El Salvador']

In [ ]:
def extract_website_details_PREVIOUS(url, cutoff=20):
    """
    Takes in a url as a string and returns the text content of the webpage, removing paragraphs shorter than cutoff words.
    Returns a list containing the title, date, and body text of the webpage.
    If the webpage cannot be accessed, returns a list of None values.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser') #creates a BeautifulSoup object from the webpage content

        #Extracts text from webpage and attempts to remove headers and footers
        extracted_text = soup.get_text(separator='\n', strip=True)
        split = extracted_text.split('\n') #split the text into a list divided every time there is a new line \n
        body = [paragraph for paragraph in split if len(paragraph.split()) > cutoff] #remove paragraphs with less than CUTOFF words
        body_text = ' '.join(body) #combine remaining text into a single string

        #Extracts title from webpage
        title = soup.title.string if soup.title else 'Title not found'

        # YOU CAN USE PYTHON's NATIVE DEBUGGER TO PAUSE CODE AT A SPECIFIC PLACE TO SEE WHAT IS HAPPENING: 
        import pdb 
        pdb.set_trace() 

        #Extracts date from webpage, if available
        #This part of the function isn't working yet, so it always returns 'Date not found'
        date = soup.find("span", class_="post-date").get_text(strip=True) if soup.find("span", class_="post-date") else 'Date not found'
        
        #Creating a BeautifulSoup object takes significant time, so this function extracts all neccessary information in one go
        #which is more efficient than creating separate functions for title, date, and body text.

        #Returns a list containing the title, date, and body text
        return [title, date, body_text]
    
    # If the webpage cannot be accessed, return a list of None values to fill in the table
    # It may be helpful to later fill in the error values instead of None to troubleshoot different problems
    except requests.RequestException as e:
        # Uncomment the next line to print errors during debugging
        #print(f"Error fetching {url}: {e}")
        return [None, None, None]
    

#this used to be a separate function, but it is now integrated into extract_website_details
#def clean_extracted_text(text, cutoff=20):
    #"""Cleans the text extracted using extract_website_text and removes short paragraphs (headers, footers, etc...) based on word count cutoff"""
    #if text is None or text == [None, None, None]:
    #    return [None, None, None]
    #else: 
    #    split = text[2].split('\n') #split the text into a list divided every time there is a new line \n
    #    body = [paragraph for paragraph in split if len(paragraph.split()) > cutoff] #remove paragraphs with less than CUTOFF words.
    #    body_text = ' '.join(body) #combine remaining text into a single string
    #    return [text[0], text[1], body_text]

#demonstrates the extract_website_details function with an example url
example_text = extract_website_details_PREVIOUS('https://www.contrapunto.com.sv/muro-colapso-tras-sismo-en-san-salvador-y-dejo-a-una-persona-herida/')
example_text

> /var/folders/yv/y95m0ts51pl7zc59py5b_ckm0000gn/T/ipykernel_62254/769742125.py(27)extract_website_details()
     25         #Extracts date from webpage, if available
     26         #This part of the function isn't working yet, so it always returns 'Date not found'
---> 27         date = soup.find("span", class_="post-date").get_text(strip=True) if soup.find("span", class_="post-date") else 'Date not found'
     28 
     29         #Creating a BeautifulSoup object takes significant time, so this function extracts all neccessary information in one go

'Muro colapsó tras sismo en San Salvador y dejó a una persona herida'
'Muro colapsó tras sismo en San Salvador y dejó a una persona herida'
['Una persona quedó atrapada tras el colapso de un muro en San Salvador, provocado por un sismo. Fue rescatada y atendida por Cruz Verde Salvadoreña.', 'Una persona quedó atrapada bajo los escombros de un muro que colapsó en la colonia Las Flores, calle Agua Caliente, San Salvador Este, tras el sismo 

### UPDATED extract_website_dtails() function 

In [ ]:
HEADERS = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome Safari"}

def extract_publish_date_from_html(soup, html_text):
    """Try JSON-LD Article/NewsArticle, meta tags, <time datetime>, else None."""
    # Try a common meta tag
    meta = soup.find("meta", attrs={"property": "article:published_time"})
    if meta and meta.get("content"):
        return meta["content"]

    # Try a simple <time> tag
    t = soup.find("time", attrs={"datetime": True})
    if t and t.get("datetime"):
        return t["datetime"]

    print("[WARN] publish date not found (skeleton).")
    return None

def parse_any_date_to_iso(date_str):
    """Return ISO-8601 (UTC) from various formats; warn if missing."""
    if not date_str:
        print("[WARN] date was not extracted.")
        return None

    # Minimal parse using pandas if available; otherwise return raw.
    try:
        import pandas as pd
        ts = pd.to_datetime(date_str, errors="coerce", utc=True)
        if pd.isna(ts):
            print(f"[WARN] could not parse date string (skeleton): {date_str!r}")
            return None
        return ts.isoformat()
    except Exception as e:
        print(f"[WARN] basic parse unavailable ({e}); returning raw date string.")
        return date_str

def extract_website_details(url, cutoff=20):
    """
    Fetch URL and return [title, date_iso, body_text].
    Keeps only paragraphs with > cutoff words.
    """
    try:
        resp = requests.get(url, headers= HEADERS, timeout=20)
        resp.raise_for_status()
    except requests.RequestException as e:
        print(f"[WARN] Request failed for {url}: {e}")
        return [None, None, None]

    soup = BeautifulSoup(resp.content, "html.parser")
    html_text = resp.text

    # title with fallback
    title = (soup.title.string.strip() if soup.title and soup.title.string else None)
    if not title:
        og = soup.find("meta", attrs={"property":"og:title"}) or soup.find("meta", attrs={"name":"title"})
        title = og["content"].strip() if og and og.get("content") else "Title not found"
        if title == "Title not found": print("[WARN] title was not extracted.")

    # body text (simple but reliable)
    parts = [p.strip() for p in soup.get_text(separator="\n", strip=True).split("\n") if p.strip()]
    body = " ".join([p for p in parts if len(p.split()) > cutoff]).strip()
    if not body:
        print("[WARN] body text empty after filtering; consider lowering cutoff.")

    # date
    raw_date = extract_publish_date_from_html(soup, html_text)
    iso_date = parse_any_date_to_iso(raw_date)
    return [title, iso_date, body]

In [6]:
def urls_to_table(urls):
    """Extracts text list of urls returns a table filled with urls, article titles, and body text."""
    
    url_title_date_text = [extract_website_details(url) for url in urls]

    cleaned_texts = [text[2] for text in url_title_date_text]
    dates = [text[1] for text in url_title_date_text]
    titles = [text[0] for text in url_title_date_text]

    #[clean_extracted_text(text, section='header') for text in raw_url_text]
    #cleaned_texts = [clean_extracted_text(text, section='body') for text in raw_url_text]

    extracted_info = pd.DataFrame({
        'url': urls,
        'title': titles,
        'date': dates,
        'body': cleaned_texts
    })
    
    return extracted_info

#demonstrates the function with the first file in the folder
urls_1 = urls_in_file('eml_files/Google Alert - _Bitcoin_ _El Salvador__1.eml')
mineria_1 = urls_to_table(urls_1)
mineria_1

,url,title,date,body
0,https://cointelegraph.com/learn/articles/bitco...,None,None,None
1,https://tradersunion.com/news/editors-picks/sh...,None,None,None
2,https://www.novinite.com/articles/231545/The%2...,The Role of Cryptocurrency in Developing Econo...,Date not found,"Money, for most people, is something they don’..."
3,https://blockchain.news/flashnews/trump-to-mee...,Trump to Meet El Salvador Leader at White Hous...,Date not found,"According to @rovercrc, former President Trump..."
4,https://crypto.news/trump-plans-white-house-vi...,Trump plans White House visit for El Salvador’...,Date not found,"by Bloomberg, follows Bukele’s agreement to de..."
5,https://www.crypto-reporter.com/press-releases...,"Bukele Rejects IMF, Keeps Buying BTC, and FXGu...",Date not found,El Salvador’s President Nayib Bukele made news...
6,https://cryptobriefing.com/bitcoin-meeting-whi...,Trump plans to meet with Bitcoin bull Nayib Bu...,Date not found,The two pro-Bitcoin leaders have maintained co...
7,https://www.cryptoninjas.net/news/new-legislat...,New Legislation Introduced in Panama to Turn t...,Date not found,Panama is set to introduce a comprehensive dra...
8,https://news.bitcoin.com/first-tokenized-wareh...,First Tokenized Warehouse Complex Built in El ...,Date not found,
9,https://coincentral.com/trump-to-host-nayib-bu...,Trump to Host Nayib Bukele Following Deportati...,Date not found,Trump plans to host El Salvador's President Bu...


In [7]:
def alert_folder_to_table(file_path):
    """Extracts text from all .eml files in a folder and returns a table filled with urls, article titles, and body text."""
    
    urls = [url for url in urls_in_folder(file_path)]

    return urls_to_table(urls)

#demonstrates the function with the full mineria google alert folder

#takes 12-18 minutes to run it may be helpful to comment out
example_table = alert_folder_to_table('eml_files/1. El Salvador Mineria 1-94 eml')
example_table[:3]

KeyboardInterrupt: 

In [9]:
#exports example_table to a csv file
#example_table.to_csv('export_csv_files/example_table.csv', index=False)

In [ ]:
def extract_keywords_from_text_PREVIOUS(text, language='english', ignore=[]):
    """ 
    Extracts keywords from a given text, ignoring specified keywords.
    Not working yet... so far just kind of spews nonsense for some reason
    """
    rake = Rake(language=language)
    rake.extract_keywords_from_text(text)
    top_phrases = rake.get_ranked_phrases()
    return top_phrases

trial_url = urls_in_file('eml_files/Google Alert - _Bitcoin_ _El Salvador__1.eml')[4]
trial_text = extract_website_details(trial_url)[2]
trial_keywords = extract_keywords_from_text(trial_text)
trial_keywords

['fxguys lets users trade without checking identity',
 'national asset built trust among',
 '1 listing price ozak ai',
 'president nayib bukele made news',
 'framework helps investors earn regularly',
 'yield real financial gains rather',
 'high growth potential besides bitcoin',
 'one main trait distinguishing fxguys',
 'fxguys users get immediate benefits',
 'investors seek options beyond long',
 'profits plus broker trading revenue',
 'lets users earn rewards',
 'fxguys gives traders direct chances',
 'top prop trading companies',
 'fxg tokens gives users',
 'fxguys platform gives access',
 'investors direct financial rewards',
 'price rises like bitcoin',
 'prop trading funding program',
 'offers clear financial growth',
 'rewards users actively',
 'every trade made',
 'fxguys gives traders',
 'several trade terminals',
 'international monetary fund',
 'fxg token costs',
 'el salvador sticks',
 'direct money gain',
 'demand plus liquidity',
 'trade2earn method gives',
 'el salvador

### Updated keyword extraction that uses a Count Vectorizer

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np 
import pandas as pd 

def extract_keywords_from_text(texts, top_k=50, ngram_range=(1,2), language="spanish", min_df=2, max_df=0.9):
    """
    Return top_k (term, count) from a corpus using CountVectorizer.
    """
    vec = CountVectorizer(stop_words=language, ngram_range=ngram_range, min_df=min_df, max_df=max_df)
    X = vec.fit_transform(pd.Series(texts).fillna(""))

    counts = np.asarray(X.sum(axis=0)).ravel()
    terms  = np.array(vec.get_feature_names_out())
    order  = counts.argsort()[::-1][:top_k]
    out = pd.DataFrame({"term": terms[order], "count": counts[order]}).reset_index(drop=True)
    print(f"[INFO] Extracted top {len(out)} terms.")
    return out